# Civilian Transportation Impacts and Proximity U.S. Military Bases

Sophie McDowall (Georgetown University Masters in Data Science and Analytics)  
December 17, 2025

United States airline traffic has been significantly disrupted this year due to personnel cuts and the government shutdown. Aviation is critical for transportation and continuity of supply chain functioning. Civilian infrastructure also underpins the ability of the U.S. military to mobilize forces. In my analysis, I will use BTS airline delay data to identify relationships between commercial air traffic delays and proximity to U.S. military bases. Through this analysis, I aim to understand the prioritization of flights and the areas of highest impact resulting from cuts to TSA and air traffic control earlier this year, along with the potential impact of being nearer to a military base and whether these airports are prioritized in times of extreme delays.

In [ ]:
# packages
library(tidyverse) |> suppressPackageStartupMessages()
library(sf) |> suppressPackageStartupMessages()
library(mapview) |> suppressPackageStartupMessages()
library(spatstat) |> suppressPackageStartupMessages()
library(jsonlite) |> suppressPackageStartupMessages()
library(lubridate) |> suppressPackageStartupMessages()
library(rnaturalearth) |> suppressPackageStartupMessages()
library(ggplot2) |> suppressPackageStartupMessages()
library(tibble) |> suppressPackageStartupMessages()
library(spatstat.geom) |> suppressPackageStartupMessages()
library(webshot2) |> suppressPackageStartupMessages()
library(dplyr) |> suppressPackageStartupMessages()

## 1 Introduction

The U.S. military relies on civilian infrastructure including highways, seaports, and airports to mobilize in the case of a national emergency or wartime. This research seeks to understand potential relationships between transportation incidents (accidents and delays) and proximity to military base, especially whether the length or likelihood of airline delays has any relation to military base proximity.

1.  **Are there more or less transportation incidents near military bases?**
2.  **How does proximity to military bases impact delays at civilian airports?**